In [1]:
import requests

In [2]:
import geopandas as gpd
import pandas as pd

## Open State data

In [3]:
file_path = "data/cb_2018_us_state_500k/cb_2018_us_state_500k.shp"
state_gdf = gpd.read_file(file_path)

## Get Labor Data

In [4]:
table = "S2301"

url = "https://api.census.gov/data/2023/acs/acs5/subject"
params = {
    "get": f"group({table})",
    "ucgid": "pseudo(0100000US$0400000)",
}
response = requests.get(url, params=params)

In [5]:
data = response.json()
columns = data[0]
rows = data[1:]
labor_df = pd.DataFrame(rows, columns=columns)

In [6]:
url = "http://api.census.gov/data/2023/acs/acs5/subject/variables.json"
response = requests.get(url)
variables = response.json()["variables"]

In [7]:
col_vars = {k: v["label"] for k, v in variables.items() if k.startswith(table)}

In [8]:
labor_df = labor_df.rename(columns=col_vars)
labor_df = labor_df[[*list(col_vars.values()), "ucgid"]]

In [9]:
rename_dict = {
    col: col.split("!!")[-1]
    for col in labor_df.columns
    if col.startswith(
        "Estimate!!Labor Force Participation Rate!!Population 20 to 64 years!!SEX!!Female"
    )
}

In [10]:
rename_dict

{'Estimate!!Labor Force Participation Rate!!Population 20 to 64 years!!SEX!!Female!!With own children under 18 years!!With own children under 6 years and 6 to 17  years': 'With own children under 6 years and 6 to 17  years',
 'Estimate!!Labor Force Participation Rate!!Population 20 to 64 years!!SEX!!Female!!With own children under 18 years!!With own children under 6 years only': 'With own children under 6 years only',
 'Estimate!!Labor Force Participation Rate!!Population 20 to 64 years!!SEX!!Female!!With own children under 18 years': 'With own children under 18 years',
 'Estimate!!Labor Force Participation Rate!!Population 20 to 64 years!!SEX!!Female': 'Female',
 'Estimate!!Labor Force Participation Rate!!Population 20 to 64 years!!SEX!!Female!!With own children under 18 years!!With own children 6 to 17 years only': 'With own children 6 to 17 years only'}

In [11]:
labor_df.rename(columns=rename_dict, inplace=True)

In [13]:
labor_df = labor_df[["Female", "ucgid"]]
labor_df = labor_df.rename(columns={"ucgid": "AFFGEOID"})

In [14]:
labor_df["Female"] = labor_df["Female"].astype(float)

## Merge Data

In [15]:
labor_gdf = state_gdf.merge(labor_df, on="AFFGEOID", how="inner")

In [16]:
labor_gdf = labor_gdf.to_crs(9311)
labor_gdf.to_file("data/Labor_Participation_Rate_Per_State_Female.gpkg")